<img width="8%" alt="Growth" src="https://naasai-public.s3.eu-west-3.amazonaws.com/abi-demo/growth_marketing.png" style="border-radius: 15%">

# Growth - Create CONTACTS view

**Tags:** #growth  #googlesheets #gsheet #data #naas_drivers #people #organizations #contacts

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook updates your contacts view in Google Sheets.

## Input

### Import libraries

In [ ]:
from naas_drivers import gsheet, linkedin
import pandas as pd
import os
from datetime import date, datetime
import naas_data_product

### Setup variables
**Inputs**
- `entity_index`: Entity index.
- `entity_dir`: Entity directory.
- `input_dir`: Input directory to retrieve file from.
- `file_people`: Name of the file storing people data to be retrieved.
- `file_organizations`: Name of the file storing organization data to be retrieved.
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_contacts`: Google Sheets sheet name storing leads profiles.

**Outputs**
- `output_dir`: Output directory to save file to.
- `file_contacts`: Output file name to save as picke.

In [ ]:
# Inputs
entity_index = "0"
entity_dir = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", entity_index), "entity_dir")
input_dir = os.path.join(entity_dir, "growth-engine", date.today().isoformat())
file_people = "people"
file_organizations = "organizations"
file_interactions = "interactions"
spreadsheet_url = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", entity_index), "abi_spreadsheet")
sheet_contacts = "CONTACTS"
api_key = os.environ.get("NAAS_API_TOKEN") or naas.secret.get('NAAS_API_TOKEN')
linkedin_url = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", entity_index), "linkedin_url")

# Outputs
output_dir = os.path.join(entity_dir, "growth-engine", date.today().isoformat())
file_contacts = "contacts"
datalake_dir = naas_data_product.OUTPUTS_PATHdatalake_dir = naas.secret.get("ABI_DATALAKE_DIR")
entity_name = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", entity_index), "entity_name") or ""

## Model

### Get contacts

In [ ]:
df_init = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_contacts)
if not isinstance(df_init, pd.DataFrame):
    df_init = pd.DataFrame()
    messaging_options = {}
else:
    messaging_options = get_dict_from_df(df_init, "MESSAGING_OPTIONS", "PEOPLE_PROFILE_URL", "messaging_options", output_dir)
print("- Contacts (init):", len(df_init))
df_init.head(3)

### Get people

In [ ]:
df_people = pload(input_dir, file_people)    
print('- People:', len(df_people))
df_people.head(1)

### Get organizations

In [ ]:
df_organizations = pload(input_dir, file_organizations)    
print('- Organizations:', len(df_organizations))
df_organizations.head(1)

### Get entity organization

In [ ]:
def get_entity_org(
    linkedin_url,
    datalake_dir
):
    # Init
    organization = None
    
    # Get data
    if "/in/" in linkedin_url:
        linkedin_dir = os.path.join(datalake_dir, "datalake", "linkedin", "profiles")
        df = get_linkedin_data(linkedin_url, linkedin_dir, "top_card")
    else:
        linkedin_dir = os.path.join(datalake_dir, "datalake", "linkedin", "organizations")  
        df = get_linkedin_data(linkedin_url, linkedin_dir, "company_info")
    if len(df) > 0:
        organization = df.loc[0, "COMPANY_NAME"]
    return organization

organization = get_entity_org(linkedin_url, datalake_dir)
print("- Entity URL:", linkedin_url)
print("- Organization:", organization)

### Get prompt sales messagings

In [ ]:
df_gsheet = gsheet.connect(spreadsheet_url).get(sheet_name="ENTITY").fillna("NA")
df_gsheet = df_gsheet[df_gsheet["LINKEDIN_URL"] == linkedin_url].reset_index(drop=True)
prompt_sales_messagings = df_gsheet.loc[0, "PROMPT_SALES_MESSAGINGS"]
print("- Prompt sales messagings:", prompt_sales_messagings)

### Get interactions

In [ ]:
df_interactions = pload(input_dir, file_interactions)    
print('- Interactions:', len(df_interactions))
df_interactions.head(3)

### Create contact view

In [ ]:
def get_metadata_by_profile(
    df_init,
    people
):
    # Init
    df = df_init.copy()
    notes = {}
    
    # Cleaning
    df = df[df["PROFILE_URL"].isin(people)]
    to_select = [
        "ENTITY",
        "INTERACTION_DATE",
        "PROFILE_URL",
        "FULLNAME",
        "CONTENT_TITLE",
        "CONTENT_URL",
        "TYPE",
        "CONTENT",
    ]
    df = df[to_select].sort_values(by="PROFILE_URL").reset_index(drop=True)
    df["INTERACTION_TEXT"] = ""
    df.loc[df["TYPE"] == "POST_REACTION", "INTERACTION_TEXT"] = df["FULLNAME"]  + " sent '" + df["CONTENT"].str.lower() + "' reaction to '" + df["CONTENT_TITLE"].str.strip() + "' (" + df["CONTENT_URL"] + ") on " + df["INTERACTION_DATE"].astype(str)
    df.loc[df["TYPE"] == "POST_COMMENT", "INTERACTION_TEXT"] = df["FULLNAME"]  + " commented '" + df["CONTENT"].str.capitalize() + "' on '" + df["CONTENT_TITLE"].str.strip() + "' (" + df["CONTENT_URL"] + ") on " + df["INTERACTION_DATE"].astype(str)

    # Loop on people (LinkedIn URL)
    for p in people:
        tmp_df = df.copy()
        tmp_df = tmp_df[tmp_df["PROFILE_URL"] == p][:10].reset_index(drop=True)
        interactions = []
        for row in tmp_df.itertuples():
            # Append interaction text to create notes
            interaction_text = row.INTERACTION_TEXT
            interactions.append(interaction_text)
            
        # Add list to dict
        notes[p] = interactions
    return notes

def create_contacts_view(
    df_people,
    df_organizations,
    df_interactions,
    api_key,
    prompt_sales_messagings,
    messaging_options,
    entity_name,
    limit_llm=50
):
    # Entity
    df_entity = df_interactions.copy()
    df_entity = df_entity[["ENTITY", "PROFILE_URL"]].drop_duplicates("PROFILE_URL", keep="first")
    
    # People
    df1 = pd.merge(df_entity, df_people, how="inner", on="PROFILE_URL")
    df1.loc[:, "LEAD_STATUS"] = "Lead"
    df1.loc[df1["MQL_DATE"] != "NA", "LEAD_STATUS"] = "Marketing Qualified Lead"
    df1.loc[df1["SQL_DATE"] != "NA", "LEAD_STATUS"] = "Sales Qualified Lead"
    to_keep = [
        'ENTITY',
        'ID',
        'FULLNAME',
        'OCCUPATION',
        'SENIORITY',
        'DEPARTMENT',
        'INTERACTION_SCORE',
        "LEAD_STATUS",
        'CREATED_DATE',
        'MQL_DATE',
        'SQL_DATE',
        'LAST_INTERACTION_DATE',
        'LAST_INTERACTION_DAYS',
        'PROFILE_URL',
        'CRM_CONTACT_ID',
        'ORGANIZATION',
    ]
    df1 = df1[to_keep]
    for c in to_keep:
        if "DATE" not in c and c not in ['ENTITY', "LEAD_STATUS", "ORGANIZATION", "CRM_CONTACT_ID"]:
            df1 = df1.rename(columns={c: f"PEOPLE_{c}"})
    
    # Organizations
    df2 = df_organizations.copy()
    to_keep = [
        'ORGANIZATION',
        'INDUSTRY',
        'CITY',
        'COUNTRY',
        'STAFF_RANGE_NAME',
        'STAFF_COUNT',
        'FOLLOWERS_COUNT',
        'INTERACTION_SCORE',
        'WEBSITE',
        'TAGLINE',
        'DESCRIPTION',
        'ORG_NAME',
        'ORG_LINKEDIN_ID',
        'ORG_LINKEDIN_URL',
        'CRM_ORG_ID'
    ]
    df2 = df2[to_keep]
    for c in to_keep:
        if not c.startswith("ORG") and c not in ["CRM_ORG_ID"]:
            df2 = df2.rename(columns={c: f"ORG_{c}"})
    
    # Merge dfs
    df = pd.merge(df1, df2, how="left").fillna('NA')
    df = df.sort_values(by="LAST_INTERACTION_DATE", ascending=False)
    
    # Exclude entity org
    df = df[df["ORGANIZATION"] != organization].reset_index(drop=True)
    
    # Add messagings options to df
    df["MESSAGING_OPTIONS"] = df["PEOPLE_PROFILE_URL"].map(messaging_options).fillna('TBD')
    
    # Filter data
    filter_df = df[
        (df["MESSAGING_OPTIONS"] == "TBD") &
        (df["LEAD_STATUS"] == "Sales Qualified Lead") & 
        (~df["PEOPLE_PROFILE_URL"].isin(list(messaging_options.keys())))
    ]
    print("-> New Sales Qualified Leads:", len(filter_df))
    # Create notes from interactions
    profiles = filter_df["PEOPLE_PROFILE_URL"].unique()
    notes = get_metadata_by_profile(df_interactions, profiles)
    filter_df["PEOPLE_NOTES"] = filter_df["PEOPLE_PROFILE_URL"].map(notes)

    # Generate messaging options for SQL
    col_messaging = [
        "PEOPLE_PROFILE_URL",
        "LAST_INTERACTION_DATE",
        "PEOPLE_FULLNAME",
        "PEOPLE_OCCUPATION",
        "PEOPLE_SENIORITY",
        "PEOPLE_DEPARTMENT",
        "PEOPLE_NOTES",
        "ORG_INDUSTRY",
        "ORG_COUNTRY",
    ]
    data = filter_df[col_messaging]
    prompt_messaging = prompt_sales_messagings + """
    The options should be presented as follow: 1.xxxx, 2.xxxx, 3.xxxx,
    The data you will received will be structured as follow:
    - LAST_INTERACTION_DATE: last interaction date with content published by [ENTITY]
    - PEOPLE_FULLNAME: Name of the person
    - PEOPLE_OCCUPATION: Job position
    - PEOPLE_SENIORITY: Job seniority
    - PEOPLE_DEPARTMENT: Job department
    - PEOPLE_NOTES: Interactions made by the person on the content published
    - ORGANIZATION: Name of the organization the person is working with
    - ORG_INDUSTRY: Company industry
    """
    prompt_messaging = prompt_messaging.replace("[ENTITY]", entity_name)
    
    count = 1
    for row in filter_df.itertuples():
        index = row.Index
        fullname = row.PEOPLE_FULLNAME
        profile_url = row.PEOPLE_PROFILE_URL
        lead_status = row.LEAD_STATUS
        messaging = "TBD"
        if count <= limit_llm:
            print(f"{count} - Messaging for:", fullname)
            tmp_df = data[data["PEOPLE_PROFILE_URL"] == profile_url].reset_index(drop=True)
            try:
                messaging = create_chat_completion(api_key, prompt_messaging, str(tmp_df.to_dict()))
            except Exception as e:
                print(e)
            if str(messaging) == "None":
                messaging = "TBD"
                print("Messaging options:", messaging)
            print()
            messaging_options[profile_url] = messaging
            pdump(output_dir, messaging_options, "messaging_options")
            df.loc[index, "MESSAGING_OPTIONS"] = messaging
            count += 1
    return df.reset_index(drop=True)

df_contacts = create_contacts_view(
    df_people,
    df_organizations,
    df_interactions,
    api_key,
    prompt_sales_messagings,
    messaging_options,
    entity_name
)
df_contacts.head(1)

## Output

### Save data

In [ ]:
pdump(output_dir, df_contacts, file_contacts)

### Send "Contacts" to spreadsheet

In [ ]:
send_data_to_gsheet(df_contacts, df_init, spreadsheet_url, sheet_contacts)

### Save table data

In [ ]:
save_data(df_contacts, datalake_dir, entity_name, file_contacts)